In [1]:
from requests import get, post
from web3 import Web3, HTTPProvider
from icecream import ic
import numpy as np
import pandas as pd
import sys
from dotenv import load_dotenv
import os
import math
import json

In [2]:
# Add the directory containing the module to the Python path
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Import functions from the module
from uniV3Pricing import get_account_data, get_uniswap_slot0, get_nft_positions_details, get_amounts_from_ticks

In [3]:
# Explicitly specify the path to .env file using forward slashes
load_dotenv(dotenv_path='.env')

# Verify that the environment variable is loaded correctly
alchemy_key = os.getenv('ALCHEMY_KEY')

# Create the Web3 provider using the Alchemy URL
w3 = Web3(HTTPProvider(f"https://base-mainnet.g.alchemy.com/v2/{alchemy_key}")) # BASE
# w3 = Web3(HTTPProvider(f"https://eth-mainnet.g.alchemy.com/v2/{alchemy_key}")) # ETHEREUM

# Check connection
print(f"Web3 is connected: {w3.is_connected()}")


Web3 is connected: True


Example Pool: 
          
- `"name": "wETH-USDC",`
- `"pool": "0xd0b53D9277642d899DF5C87A3966A349A798F224",`
- `"token0": "0x4200000000000000000000000000000000000006",`
- `"token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",`
- `"decimal0": 18,`
- `"decimal1": 6`


Key Functions:
- `sqrt_price_x96_to_price`: Converts the square root price x96 to a price.
- `price_to_tick`: Converts a price to a tick value.
- `get_account_data`: Retrieves account data based on the provided address.
- `get_uniswap_slot0`: Fetches slot 0 data from the Uniswap pool.
- `get_nft_positions_details`: Retrieves NFT position details for a given account.
- `get_amounts_from_ticks`: Calculates amounts based on tick values and liquidity.

Custom Pricing Strategy:
- The notebook demonstrates a custom pricing strategy based on predefined token prices in USD.
- It adjusts the token prices based on decimal values and calculates a custom tick value using the price_to_tick function.

In [4]:
def sqrt_price_x96_to_price(sqrt_price_x96):
    Q96 = 0x1000000000000000000000000
    return (sqrt_price_x96 / Q96) ** 2


def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

In [5]:
# Current status of the pool:

pool_dict = {
            "name": "wETH-USDC",
            "pool": "0xd0b53D9277642d899DF5C87A3966A349A798F224",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "decimal0": 18,
            "decimal1": 6
            }

pool_data = get_uniswap_slot0(pool_dict["pool"], w3)
print(f'Pool Data: {json.dumps(pool_data, indent=4)}')

sqrtPriceX96 = pool_data['sqrtPriceX96']


price = sqrt_price_x96_to_price(sqrtPriceX96) * 10**(pool_dict["decimal0"]) * 10**(-pool_dict["decimal1"])
print(f'Price of token0 in token1 after decimal adjustment: {price}')


Pool Data: {
    "sqrtPriceX96": 4417273057366869622785059,
    "tick": -195902,
    "observationIndex": 1289,
    "observationCardinality": 2000,
    "observationCardinalityNext": 2000,
    "feeProtocol": 0,
    "unlocked": true
}
Price of token0 in token1 after decimal adjustment: 3108.488931020529


In [6]:
# Custom Pricing

token0_price_in_usd = 3080.00
token1_price_in_usd = 0.99

token0_in_token1 = token0_price_in_usd / token1_price_in_usd
print(f'Price of token0 in token1: {token0_in_token1}')

# Decimal adjustments

token0_in_token1_adjusted = token0_in_token1 * 10**(-pool_dict["decimal0"]) * 10**(pool_dict["decimal1"])
print(f'Price of token0 in token1 adjusted: {token0_in_token1_adjusted}')

custom_tick = price_to_tick(token0_in_token1_adjusted)
print(f'Custom tick: {custom_tick}')

Price of token0 in token1: 3111.1111111111113
Price of token0 in token1 adjusted: 3.1111111111111114e-09
Custom tick: -195893


In [7]:
# AssetDetails of 0xb2DF87b16682435bdf29494fEA44dF5374F22f3E account
account_data = get_account_data('0xb2DF87b16682435bdf29494fEA44dF5374F22f3E', w3)


print(json.dumps(account_data, indent=4))

[
    [
        "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
        "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
        "0x4200000000000000000000000000000000000006",
        "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
        "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1"
    ],
    [
        0,
        0,
        0,
        720767,
        720766
    ],
    [
        143866670,
        422153165,
        44815470303526491,
        1,
        1
    ]
]


In [8]:
nft_data = get_nft_positions_details(account_data[0][4], w3, account_data[1][4])
print(json.dumps(nft_data, indent=4))

{
    "nonce": 0,
    "operator": "0x0000000000000000000000000000000000000000",
    "token0": "0x4200000000000000000000000000000000000006",
    "token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
    "fee": 500,
    "tickLower": -196610,
    "tickUpper": -195600,
    "liquidity": 13057076758385779,
    "feeGrowthInside0LastX128": 41610421791424474659770915678114485696014,
    "feeGrowthInside1LastX128": 126828130941184306312069351412926,
    "tokensOwed0": 0,
    "tokensOwed1": 0
}


In [9]:
get_amounts_from_ticks(custom_tick, tickLower=nft_data["tickLower"], tickUpper=nft_data["tickUpper"], liquidity=nft_data["liquidity"])

(3404352141796071324, 25644990651)